In [24]:
import geopy.distance
import pandas as pd
import regex as re
import requests
import os.path
import random
import math

# defaults
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
adjacency_path = os.path.join(parent_directory, "adjacency.csv")

# global vars
counties = {}
detailed = False

# input
start_locations = ['Red, Washington County, UT'] # , 'Red, Fairfield County, CT'
goal_locations = ['Red, San Diego County, CA'] # , 'Red, Rensselaer County, NY'

### classes   
class County:
    def __init__(self, txt:str):
        self.name = txt.split(",")[0]
        self.state = txt.split(",")[1]
        self.id = txt
        self.neighbors = []
        self.parent = None
        self.visited = False
        self.color = ''

        self.g = float('inf') # distance from start
        self.h = float('inf') # heuristic distance from the goal
        self.f = 0
    
    def add_neighbor(self, neighbor):
        if f"{self.name}, {self.state}" != f"{neighbor.name}, {neighbor.state}":
            self.neighbors.append(neighbor)

    def update_f(self, value = None):
        if value is None:
            self.f = self.g + self. h
        else: self.f = value
### end block classes

### functions
def read_neighbors_file(file_name: os) -> pd.DataFrame: # reads the csv file and converts it into a dataframe
    return pd.read_csv(file_name)

def get_unique_list(df: pd.DataFrame, col_name: str) -> list: # returns the unique values in a df[col_name]
    return list(set(df[col_name]))

def make_object_list(lst: list) -> list: # gets a list of text and returns a list of County objects
    return [County(c) for c in lst]

def preparing_objects(raw_df: pd.DataFrame) -> dict: # making the dataframe into objects and adding their neighbors
    unique_counties = get_unique_list(raw_df, 'countyname')
    county_objects = make_object_list(unique_counties) 
    counties_dict = {county.name + "," + county.state: county for county in county_objects}
    for _, record in raw_df.iterrows():
        county = record['countyname']
        neighbor = record['neighborname']
        cnty_object = counties_dict[county]
        neighbor_object = counties_dict[neighbor]
        cnty_object.add_neighbor(neighbor_object)
    return counties_dict

def find_path(starting_locations, goal_locations, search_method, detail_output): # finds the shortest path from the starting locations to the goal location using a search method
    global detailed 
    detailed = detail_output
    pathes = []
    for start in starting_locations:
        if search_method == 1:
            path = a_star(start, goal_locations)
        elif search_method == 2:
            path = hill_climbing(start, goal_locations, 0) # 0 as it is the first iteration
        elif search_method == 3:
            tempature = 100
            path = simulated_annealing(start,goal_locations, (tempature if tempature <=100 else 100))
        elif search_method == 4:
            path = local_beam(start, goal_locations)
        # adding the path to the paths list
        if not path:
            pathes.append('No path found.')
        else: pathes.append(path)
    return pathes

def get_county_coordinates(county_name): # returns the minimum distance between a start county and one of the goals
    url = f"https://nominatim.openstreetmap.org/search?q={county_name}&format=json"
    headers = {
        'User-Agent': 'YourAppName/1.0 (your.email@example.com)'
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        if data:
            return float(data[0]['lat']), float(data[0]['lon'])
        else:
            raise Exception(f"No data found for {county_name}")
    else:
        raise Exception(f"Problem with the API. Status code: {response.status_code}")

def heuristic_calc(start, goal): # returns the distance between two coordinates in km
    start_cord = get_county_coordinates(start)
    goal_cord = get_county_coordinates(goal)
    distance = geopy.distance.geodesic(start_cord, goal_cord).km
    return distance

def retracePath(c, path=None):
    if path is None:
        path = []
    if c.parent:
        retracePath(c.parent, path)
    path.append(c.id)
    c.visited = True
    return path

def a_star(start, goals): # performs A* search from a starting location to one of the ending locations in the goal list
    frontier = [] # have been visited but not expanded 
    explored = set() # visited and expanded
    path = []
    start_county = counties[start]
    start_county.g = 0
    start_county.h = min(heuristic_calc(start_county.id, counties[goal].id) for goal in goals)
    start_county.update_f()
    frontier.append(counties[start])

    while frontier:
        current = min(frontier, key = lambda county: county.f) # takes the county with the minimum f
        if current.id in goals and not current.visited and current.color == start_county.color : # reached an unvisited goal node
            path = retracePath(current)
            return path
        frontier.remove(current)
        current.visited = True
        explored.add(current)
        for neighbor in current.neighbors:
            if neighbor in explored: continue
            tentative_g = current.g + heuristic_calc(current.id, neighbor.id)
            if tentative_g < neighbor.g:
                neighbor.parent = current
                neighbor.g = tentative_g
                neighbor.h = min(heuristic_calc(neighbor.id, counties[goal].id) for goal in goals)
                neighbor.update_f()

            if neighbor not in frontier:
                frontier.append(neighbor)
    return path

def hill_climbing(start, goals, iteration, max_iteration = 4): # performs a hill-climbing search to find the global maximum which is one of the goal nodes
    if iteration == max_iteration: # after 5 iterations the hill climbing should be stopped
        return None
    current = counties[start]
    path = []
    while True:
        if (current.id in goals) and (not current.visited) and (current.color == counties[start].color) : # reached an unvisited goal node
            path = retracePath(current)
            return path
        else:
            neighbors = current.neighbors
            next_neighbor = min(neighbors, key=lambda neighbor: min(heuristic_calc(neighbor.id, goal) for goal in goals))
            if min(heuristic_calc(next_neighbor.id, goal) for goal in goals) >= min(heuristic_calc(current.id, goal) for goal in goals):
                return hill_climbing(random.choice(neighbors).id, goals, iteration + 1)
            next_neighbor.parent = current
            current = next_neighbor
            if iteration == max_iteration:
                break
    return path

def simulated_annealing(start, goals, temp): # PROBLEM
    def temperature(k):
        return 1 - ((k + 1) / temp)

    def choose_random_neighbor(neighbors):
        valid_neighbors = [neighbor for neighbor in neighbors if not neighbor.visited]
        if not valid_neighbors:
            return None
        neighbor = random.choice(valid_neighbors)
        neighbor.visited = True
        return neighbor 

    current = counties[start]
    current.visited = True
    path = []
    for k in range(temp):
        T = temperature(k)
        neighbor = choose_random_neighbor(current.neighbors)
        if neighbor is None:
            break
        print(f'\niteration: {k}')
        print(f'current node: {current.id}')
        print(f'neighbors: ' + str([n.id for n in current.neighbors]))
        print(f'T: {T}\nrandom neighbor: {neighbor.id}')
        if neighbor.id in goals and neighbor.color == current.color:
            path = retracePath(current)
            return path
    
        delta = min(heuristic_calc(neighbor.id, goal) for goal in goals) - min(heuristic_calc(current.id, goal) for goal in goals)
        if delta > 0:
            current = neighbor
        elif math.exp(delta / T) >= random.uniform(0, 1):
            current = neighbor
    return path

def local_beam(start, goals, k=3): # performs a k-beam search with k=3 with jumptracking
    def get_k_most_close_nodes(current_node):
        current_neighbors = current_node.neighbors 
        for neighbor in current_neighbors:
            neighbor.update_f(min(heuristic_calc(neighbor.id, goal) for goal in goals))
        return sorted(current_neighbors, key=lambda x: x.f)[:k] # sorted by distance from goal and only the top k (closest)

    current = counties[start] # initiallize the current node
    explored = set() # visited and expanded
    frontier = [current] # visited but not expanded
    while frontier:
        frontier.sort(key=lambda x: x.f)
        next = frontier.pop(0)
        if (next.id in goals) and (not next.visited) and (next.color == counties[start].color) : # reached an unvisited goal node
            return retracePath(next)
        next.visited = True
        next_neighbors = get_k_most_close_nodes(next)
        print(f'current node: {next.id}')
        print(f'current neighbors: {str([(n.id, n.f) for n in next_neighbors])}')

        if frontier and frontier[0].f < current.f:
            frontier[0].parent = current

        for neighbor in next_neighbors:
            if neighbor not in explored and neighbor not in frontier:
                neighbor.parent = current
                frontier.append(neighbor)
        # mark the current node as expanded
        explored.add(current)
    return None

def get_list_per_color(lst, pattern): # returns a list for the same color
    return [loc.replace('Red, ', '').replace('Blue, ', '') for loc in lst if re.search(pattern, loc)]

def print_paths(paths):
    # Format the paths with color indications
    all_lists = [
        [f"{county} (R)" if counties[county].color == 'Red' else f"{county} (B)" for county in path] if path != "No path found." else ["No path found."]
        for path in paths
    ]

    # Determine the maximum length of the lists
    max_length = max(len(lst) for lst in all_lists)
    previous_step = None

    # Iterate through the steps and print accordingly
    for i in range(max_length):
        step_elements = []
        for lst in all_lists:
            if i < len(lst):
                step_elements.append(lst[i])
            else:
                step_elements.append(lst[-1])  # Repeats the last element if the list is shorter

        print(f"{{{' ; '.join(step_elements)}}}")
        
        if detailed == 1 and i == 1:
            heuristics = []
            for current, prev in zip(step_elements, previous_step):
                if "No path found." in [current, prev]:
                    heuristics.append("N/A")
                else:
                    current_loc = current.split(' (')[0]
                    prev_loc = prev.split(' (')[0]
                    heuristic_value = heuristic_calc(prev_loc, current_loc)
                    heuristics.append(f"{heuristic_value:.2f}")
            print(f"Heuristic: {{{' ; '.join(heuristics)}}}")

        previous_step = step_elements

def assigning_colors_to_counties(starts, goals): # assigning color to each input location
    global start_locations, goal_locations
    for location in [*starts, *goals]:
        color, county_name, state = location.split(", ")
        county_id = f"{county_name}, {state}"
        if county_id in counties:
            counties[county_id].color = color

    # re-arranging the input lists to be without the colors
    start_locations_no_color = [location.split(', ', 1)[1] for location in starts] # temp
    goal_locations_no_color = [location.split(', ', 1)[1] for location in goals] # temp
    start_locations = start_locations_no_color
    goal_locations = goal_locations_no_color

### functions 
if __name__ == "__main__":
    raw_df = read_neighbors_file(adjacency_path)
    counties = preparing_objects(raw_df) # dict: {county.name, county.state: county object}. this is the same dict as neighbors so it is enough for one of them    
    assigning_colors_to_counties(start_locations, goal_locations)
    pathes = find_path(start_locations, goal_locations, 4, 0) # start_location, goal_locations, search_method, detailed_output
    print_paths(pathes) 